In [1]:
import sqlite3
import os

In [12]:
# Fermez la connexion à la base de données
conn.close()

In [4]:
if not os.path.isfile('olist.db'):
    print("Le fichier olist.db n'a pas été trouvé dans le répertoire courant.")
    exit()

conn = sqlite3.connect('olist.db')
cursor = conn.cursor()

# Exploration de la bdd

In [9]:
# Obtenez la liste des tables dans la base de données
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

# Explorez chaque table
for table in tables:
    print(f"\nTable '{table}':")

    # Obtenez les noms des colonnes
    cursor.execute(f"PRAGMA table_info({table});")
    columns = [col[1] for col in cursor.fetchall()]
    print("Colonnes :", ", ".join(columns))
    
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    lenght = cursor.fetchall()[0][0]
    print(f"Nombre de lignes : {lenght}")

    # Obtenez les premières lignes de la table
    cursor.execute(f"SELECT * FROM {table} LIMIT 5;")
    rows = cursor.fetchall()
    print("\nPremières lignes :")
    for row in rows:
        print(row)
        pass  
    print('\n#######################')


Table 'customers':
Colonnes : index, customer_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state
Nombre de lignes : 99441

Premières lignes :
(0, '06b8999e2fba1a1fbc88172c00ba8bc7', '861eff4711a542e4b93843c6dd7febb0', 14409, 'franca', 'SP')
(1, '18955e83d337fd6b2def6b18a428ac77', '290c77bc529b7ac935b93aa66c333dc3', 9790, 'sao bernardo do campo', 'SP')
(2, '4e7b3e00288586ebd08712fdd0374a03', '060e732b5b29e8181a18229c7b0b2b5e', 1151, 'sao paulo', 'SP')
(3, 'b2b6027bc5c5109e529d4dc6358b12c3', '259dac757896d24d7702b9acbbff3f3c', 8775, 'mogi das cruzes', 'SP')
(4, '4f2d8ab171c80ec8364f7c12e35b23ad', '345ecd01c38d18a9036ed96c73b8d066', 13056, 'campinas', 'SP')

#######################

Table 'geoloc':
Colonnes : index, geolocation_zip_code_prefix, geolocation_lat, geolocation_lng, geolocation_city, geolocation_state
Nombre de lignes : 1000163

Premières lignes :
(0, 1037, -23.54562128115268, -46.63929204800168, 'sao paulo', 'SP')
(1, 1046, -23.54608112703553, -4

# Requêtes

In [11]:
cursor.execute("""
SELECT COUNT(DISTINCT customer_unique_id) AS unique_customers
FROM customers;
""")
print(f"Proportion de clients qui ont fait plus d'1 commande : {((1 - cursor.fetchall()[0][0]/99441)*100):.2f} %")

Proportion de clients qui ont fait plus d'1 commande : 3.36 %


In [7]:
# List modes de paiement

cursor.execute("""SELECT DISTINCT payment_type FROM order_pymts""")
cursor.fetchall()

[('credit_card',), ('boleto',)]

In [8]:
# List scores avis

cursor.execute("""SELECT DISTINCT review_score FROM order_reviews""")
cursor.fetchall()

[(4,), (5,), (2,), (3,), (1,)]

In [9]:
# List types de status de commande

cursor.execute("""SELECT DISTINCT order_status FROM orders""")
cursor.fetchall()

[('delivered',)]

In [10]:
# List types de status de commande

cursor.execute("""SELECT DISTINCT product_category_name FROM products""")
cursor.fetchall()

[('perfumaria',),
 ('artes',),
 ('esporte_lazer',),
 ('bebes',),
 ('utilidades_domesticas',),
 ('instrumentos_musicais',),
 ('cool_stuff',),
 ('moveis_decoracao',),
 ('eletrodomesticos',),
 ('brinquedos',),
 ('cama_mesa_banho',),
 ('construcao_ferramentas_seguranca',),
 ('informatica_acessorios',),
 ('beleza_saude',),
 ('malas_acessorios',),
 ('ferramentas_jardim',),
 ('moveis_escritorio',),
 ('automotivo',),
 ('eletronicos',),
 ('fashion_calcados',),
 ('telefonia',),
 ('papelaria',),
 ('fashion_bolsas_e_acessorios',),
 ('pcs',),
 ('casa_construcao',),
 ('relogios_presentes',),
 ('construcao_ferramentas_construcao',),
 ('pet_shop',),
 ('eletroportateis',),
 ('agro_industria_e_comercio',),
 (None,),
 ('moveis_sala',),
 ('sinalizacao_e_seguranca',),
 ('climatizacao',),
 ('consoles_games',),
 ('livros_interesse_geral',),
 ('construcao_ferramentas_ferramentas',),
 ('fashion_underwear_e_moda_praia',),
 ('fashion_roupa_masculina',),
 ('moveis_cozinha_area_de_servico_jantar_e_jardim',),
 ('in

In [11]:
# List types de status de commande

cursor.execute("""SELECT COUNT(DISTINCT customer_unique_id) FROM customers""")
cursor.fetchall()[0][0]

100

In [12]:
# Date de la dernière commande

cursor.execute("""
SELECT MAX(order_purchase_timestamp) AS last_order_date
FROM orders;
""")

# Récupérez les résultats de la première requête
last_order_date = cursor.fetchall()[0][0].split()[0]
last_order_date

'2018-08-18'

In [5]:
# Date de la première commande

cursor.execute("""
SELECT MIN(order_purchase_timestamp) AS first_order_date
FROM orders;
""")

# Récupérez les résultats de la première requête
first_order_date = cursor.fetchall()[0][0].split()[0]
first_order_date

'2016-09-04'

### 1. En excluant les commandes annulées, quelles sont les commandes récentes de moins de 3 mois que les clients ont reçues avec au moins 3 jours de retard ?

In [10]:
req1 = f"""
SELECT order_id, julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date) AS retard
FROM orders
WHERE order_status = 'delivered'
-- AND order_purchase_timestamp >= DATE('now', '-3 months')
AND order_purchase_timestamp >= DATE({last_order_date}, '-3 months')
AND julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date) >= 3;
"""

In [11]:
cursor.execute(req1)

# Récupérez les résultats de la première requête
results1 = cursor.fetchall()

# Affichez les résultats de la première requête
print("Résultats de la commande 1 :")
for row in results1:
    print(row)

Résultats de la commande 1 :
('203096f03d82e0dffbc41ebc2e2bcfb7', 11.933171296492219)
('fbf9ac61453ac646ce8ad9783d7d0af6', 9.91937500005588)
('6ea2f835b4556291ffdc53fa0b3b95e8', 7.7912384257651865)
('a685d016c8a26f71a0bb67821070e398', 7.567546296399087)
('6a0a8bfbbe700284feb0845d95e0867f', 17.82152777770534)
('9d531c565e28c3e0d756192f84d8731f', 32.901990740559995)
('8fc207e94fa91a7649c5a5dab690272a', 32.571087962947786)
('33a3edb84b9df4cb49546859b990ac6d', 6.002696759067476)
('f31535f21d145b2345e2bf7f09d62322', 49.65942129632458)
('fb9b2930f4a30f96c7cc98eaaa85e5f0', 3.449918981641531)
('4906eeadde5f70b308c20c4a8f20be02', 6.75344907399267)
('16c24987724018f73b746aacfbb5f8e8', 7.702800925821066)
('f11e9516ca2b6091b64f2e2ea0ebebb3', 9.91995370388031)
('b60b53ad0bb7dacacf2989fe27ad567a', 5.5503124999813735)
('da8be3bb62e9bf01e2e1a3bfd74ebd1a', 4.452847222331911)
('690199d6a2c51ff57c6b392d7680cbfd', 33.520092592574656)
('7b58fca3faf09217a36d5febee5739a5', 15.877789351623505)
('f30607daae668

### 2. Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100 000 Real sur des commandes livrées via Olist ?

In [12]:
req2 = """
SELECT oi.seller_id, SUM(oi.price) AS total_sales
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY oi.seller_id
HAVING total_sales > 100000;
"""

In [13]:
cursor.execute(req2)

# Récupérez les résultats de la première requête
results2 = cursor.fetchall()

# Affichez les résultats de la première requête
print("Résultats de la commande 2 :")
for row in results2:
    print(row)

Résultats de la commande 2 :
('1025f0e2d44d7041d6cf58b6550e0bfa', 138208.55999999947)
('1f50f920176fa81dab994f9023523100', 106655.7099999974)
('46dc3b2cc0980fb8ec44634e21d2718e', 122811.38000000114)
('4869f7a5dfa277a7dca6462dcf3b52b2', 226987.92999999813)
('4a3ca9315b744ce9f8e9374361493884', 196882.11999999516)
('53243585a1d6dc2643021fd1853d8905', 217940.43999999968)
('5dceca129747e92ff8ef7a997dc4f8ca', 111126.72999999972)
('620c87c171fb2a6dd6e8bb4dec959fc6', 112461.49999999844)
('6560211a19b47992c3666cc44a7e94c0', 120702.83000000003)
('7a67c85e85bb2ce8582c35f2203ad736', 139658.69000000184)
('7c67e1448b00f6e969d365cea6b010ab', 186570.04999999996)
('7d13fca15225358621be4086e1eb0964', 112436.17999999983)
('7e93a43ef30c4f03f38b393420bc753a', 165981.49000000002)
('955fee9216a65b617aa5c0531780ce60', 131836.70999999915)
('cc419e0650a3c5ba77189a1882b7556a', 101090.96000000235)
('da8622b14eb17ae2831f4ac5b9dab84a', 159816.86999999534)
('fa1c13f2614d7b5c4749cbc52fecda94', 190917.13999999838)


### 3. Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30 produits) ?

In [14]:
req3 = f"""
SELECT sellers.seller_id, COUNT(*) as items_sold
FROM sellers
JOIN order_items ON sellers.seller_id = order_items.seller_id
JOIN orders ON order_items.order_id = orders.order_id
-- WHERE orders.order_purchase_timestamp >= DATE('now', '-3 months')
WHERE orders.order_purchase_timestamp >= DATE({last_order_date}, '-3 months')
GROUP BY sellers.seller_id
HAVING COUNT(*) > 30;
"""

In [15]:
cursor.execute(req3)

# Récupérez les résultats de la première requête
results3 = cursor.fetchall()

# Affichez les résultats de la première requête
print("Résultats de la commande 3 :")
for row in results3:
    print(row)

Résultats de la commande 3 :
('001cca7ae9ae17fb1caed9dfb1094831', 239)
('002100f778ceb8431b7a1020ff7ab48f', 55)
('004c9cd9d87a3c30c522c48c4fc07416', 170)
('00ee68308b45bc5e2660cd833c3f81cc', 172)
('00fc707aaaad2d31347cf883cd2dfe10', 135)
('0176f73cc1195f367f7b32db1e5b3aa8', 39)
('01cf7e3d21494c41fb86034f2e714fa1', 67)
('01fd077212124329bac32490e8ef80d9', 39)
('01fdefa7697d26ad920e9e0346d4bd1b', 131)
('0241d4d5d36f10f80c644447315af0bd', 246)
('02f623a8eb246f3c5f7c2f96462654e6', 44)
('04308b1ee57b6625f47df1d56f00eedf', 94)
('0432ead42b6c8a0bdf68154add917fdf', 90)
('048c2757535328e0d7dac690ad3c0aae', 51)
('0509040ea3fe50071181bbc359eb7738', 98)
('056b4ada5bbc2c50cc7842547dda6b51', 66)
('058fd0aa2bfdb2274e05e1ae971dabb6', 64)
('05d2173d43ea568aa0540eba70d2ca76', 66)
('05f51e13da97139648b8125c31e5f51b', 63)
('05ff92fedb5be47920fea08e501238b9', 62)
('062ce95fa2ad4dfaedfc79260130565f', 60)
('06579cb253ecd5a3a12a9e6eb6bf8f47', 72)
('066a6914e1ebf3ea95a216c73a986b91', 74)
('0691148aee60ca47977c

### 4. Quels sont les 5 codes postaux, enregistrant plus de 30 commandes, avec le pire review score moyen sur les 12 derniers mois ?

In [30]:
req4 = f"""
SELECT
  customers.customer_zip_code_prefix,
  COUNT(orders.order_id) AS number_of_orders,
  AVG(order_reviews.review_score) AS avg_review_score
FROM
  customers
JOIN
  orders ON customers.customer_id = orders.customer_id
JOIN
  order_reviews ON orders.order_id = order_reviews.order_id
WHERE
  orders.order_purchase_timestamp >= datetime({last_order_date}, '-12 months')
GROUP BY
  customers.customer_zip_code_prefix
HAVING
  COUNT(orders.order_id) > 30
ORDER BY
  avg_review_score ASC
LIMIT
  5;
"""

In [31]:
cursor.execute(req4)

# Récupérez les résultats de la première requête
results4 = cursor.fetchall()

# Affichez les résultats de la première requête
print("Résultats de la commande 4 :")
for row in results4:
    print(row)

Résultats de la commande 4 :
(22753, 65, 3.0307692307692307)
(22621, 33, 3.0606060606060606)
(65075, 36, 3.2777777777777777)
(28893, 37, 3.3513513513513513)
(22723, 44, 3.3636363636363638)


In [8]:
# Fermez la connexion à la base de données
conn.close()